In [6]:
import re
import pandas as pd

Data Ingestion

In [ ]:

def open_file(file_path: str):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return pd.DataFrame(columns=["id", "start_time", "end_time", "text"])
    except Exception as e:
        print(f"Error reading file: {e}")
        return pd.DataFrame(columns=["id", "start_time", "end_time", "text"])


def remove_redundunat_data(transcript):
    if transcript.startswith('WEBVTT'):
        transcript = transcript.split('\n', 1)[1] if '\n' in transcript else ''
    
    # Split transcript blocks by double newlines
    return re.split(r'\n\s*\n', transcript.strip())


#data ingestion
def convert_webvtt_to_dataframe(blocks):
    """
    Convert WEBVTT transcript file to pandas DataFrame.
    
    Args:
        file_path (str): Path to the WEBVTT file
        
    Returns:
        pd.DataFrame: DataFrame with columns [id, start_time, end_time, text]
    """
    
    records = []
    
    for block in blocks:
        if not block.strip():
            continue
            
        lines = [line.strip() for line in block.strip().split('\n') if line.strip()]
        
        # Skip if not enough lines for a valid block
        if len(lines) < 3:
            continue
            
        # Try to parse the first line as ID
        try:
            idx = int(lines[0])
        except ValueError:
            continue
        
        # Second line should be the timestamp
        time_range = lines[1]
        
        # Check if it matches timestamp pattern
        if '-->' not in time_range:
            continue
            
        # Rest of the lines are the text
        text = " ".join(lines[2:]).strip()
        
        # Extract start and end times
        try:
            start, end = re.split(r'\s*-->\s*', time_range)
            records.append([idx, start.strip(), end.strip(), text])
        except ValueError:
            print(f"Warning: Could not parse timestamp in block {idx}: {time_range}")
            continue
    
    # Convert to DataFrame - ONLY these 4 columns
    return pd.DataFrame(records, columns=["id", "start_time", "end_time", "text"])

# Simple usage
#if __name__ == "__main__":
#    df = convert_webvtt_to_dataframe('transcripts/aprilynne.txt')
#    print("Columns:", df.columns.tolist())
#    print(f"Shape: {df.shape}")
#    print("\nFirst 5 records:")
#    print(df.head())

In [8]:
transcripts = open_file('transcripts/aprilynne.txt')
blocks = remove_redundunat_data(transcripts)
df = convert_webvtt_to_dataframe(blocks)
df

,id,start_time,end_time,text
0,1,00:00:03.495,00:00:05.235,"Hey, April Lynn, how are you?"
1,2,00:00:06.395,00:00:08.275,"I am doing well, thank you."
2,3,00:00:09.575,00:00:13.635,"That's awesome. So, hi everyone. Uh, my name i..."
3,4,00:00:13.775,00:00:16.035,I'm a creator success manager at Spotter.
4,5,00:00:16.375,00:00:17.755,"We are super, super stoked"
...,...,...,...,...
1253,1254,01:10:16.065,01:10:17.885,and I hope you were able to get something usef...
1254,1255,01:10:18.525,01:10:21.295,This was so fantastic. Thank you so much for y...
1255,1256,01:10:21.555,01:10:24.415,I'm sure everybody learned so much. So thank you.
1256,1257,01:10:25.485,01:10:26.235,Thank you so much.


Data preprocessing

In [9]:
def drop_invalid_text_rows(df, text_column):
    """
    Drop rows where the text column has null, empty, or whitespace-only values.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        text_column (str): Name of the text column to check (default: 'text')
        
    Returns:
        pd.DataFrame: Cleaned DataFrame
    """
    # Remove rows with null, empty, or whitespace-only text
    cleaned_df = df[
        df[text_column].notna() & 
        (df[text_column].str.strip() != '')
    ].reset_index(drop=True)
    
    return cleaned_df

df = drop_invalid_text_rows(df, 'text')
print("Null values per column:")
print(df.isnull().sum())

print("\nEmpty strings per column:")
print((df == '').sum())

print("\nCombined (null or empty):")
print((df.isnull() | (df == '')).sum())

Null values per column:
id            0
start_time    0
end_time      0
text          0
dtype: int64

Empty strings per column:
id            0
start_time    0
end_time      0
text          0
dtype: int64

Combined (null or empty):
id            0
start_time    0
end_time      0
text          0
dtype: int64


In [10]:
def group_rows_into_splitters(df, group_size=5):
    grouped_data = []
    
    for i in range(0, len(df), group_size):
        group = df.iloc[i:i+group_size]
        
        if len(group) == 0:
            continue
        
        # Get current group's text
        current_text = ' '.join(group['text'].tolist())
        
        # Get last 10 words from previous group (if exists)
        if i > 0:  # Not the first group
            prev_group_text = grouped_data[-1]['text']  # Previous group's text
            prev_words = prev_group_text.split()
            last_10_words = ' '.join(prev_words[-10:]) if len(prev_words) >= 10 else prev_group_text
            
            # Combine: last 10 words from previous + current text
            combined_text = last_10_words + ' ' + current_text
        else:
            # First group - no previous text to add
            combined_text = current_text
            
        grouped_row = {
            'id': group['id'].tolist(),                # [1,2,3,4,5]
            'start_time': group['start_time'].iloc[0], # First start_time
            'end_time': group['end_time'].iloc[-1],    # Last end_time  
            'text': combined_text                      # Combined text with overlap
        }
        
        grouped_data.append(grouped_row)
    
    return pd.DataFrame(grouped_data)

df_grouped = group_rows_into_splitters(df, group_size=5)
print(df_grouped.head())
print(df_grouped['text'][0])
print(df_grouped['text'][1])


                     id    start_time      end_time  \
0       [1, 2, 3, 4, 5]  00:00:03.495  00:00:17.755   
1      [6, 7, 8, 9, 10]  00:00:17.755  00:00:29.195   
2  [11, 12, 13, 14, 15]  00:00:29.215  00:00:39.875   
3  [16, 17, 18, 19, 20]  00:00:39.935  00:00:53.915   
4  [21, 22, 23, 24, 25]  00:00:53.915  00:01:04.835   

                                                text  
0  Hey, April Lynn, how are you? I am doing well,...  
1  creator success manager at Spotter. We are sup...  
2  to share it with you guys, um, to introduce he...  
3  uh, and YouTube strategist who's built a name ...  
4  Um, and today she's gonna be taking us through...  
Hey, April Lynn, how are you? I am doing well, thank you. That's awesome. So, hi everyone. Uh, my name is Sie. I'm a creator success manager at Spotter. We are super, super stoked
creator success manager at Spotter. We are super, super stoked to have April Lynn on today's webinar to teach us how to make a killer intro. So a little bit ab

In [11]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

# Download required data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

df_processed = df_grouped.copy()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if pd.isna(text):
        return ""
    
    try:
        # Lowercase & tokenize
        tokens = word_tokenize(str(text).lower())
        
        clean_tokens = []
        for word in tokens:
            # Remove punctuation & non-alphabetic with regex
            word = re.sub(r'[^a-z]', '', word)
            
            if word and word not in stop_words:
                clean_tokens.append(word)
                
        return ' '.join(clean_tokens)
    except LookupError:
        return str(text).lower()

# Add processed columns
df_processed['clean_text'] = df_processed['text'].apply(clean_text)
df_processed['word_count'] = df_processed['text'].apply(
    lambda x: len(str(x).split()) if pd.notna(x) else 0
)
print(df_processed.head())

print(df_processed['text'][10])


                     id    start_time      end_time  \
0       [1, 2, 3, 4, 5]  00:00:03.495  00:00:17.755   
1      [6, 7, 8, 9, 10]  00:00:17.755  00:00:29.195   
2  [11, 12, 13, 14, 15]  00:00:29.215  00:00:39.875   
3  [16, 17, 18, 19, 20]  00:00:39.935  00:00:53.915   
4  [21, 22, 23, 24, 25]  00:00:53.915  00:01:04.835   

                                                text  \
0  Hey, April Lynn, how are you? I am doing well,...   
1  creator success manager at Spotter. We are sup...   
2  to share it with you guys, um, to introduce he...   
3  uh, and YouTube strategist who's built a name ...   
4  Um, and today she's gonna be taking us through...   

                                          clean_text  word_count  
0  hey, april lynn, how are you? i am doing well,...          34  
1  creator success manager at spotter. we are sup...          55  
2  to share it with you guys, um, to introduce he...          50  
3  uh, and youtube strategist who's built a name ...          59

Create embeddings and vectorize

In [12]:
import numpy as np
print(np.__version__)
from sentence_transformers import SentenceTransformer
# %pip install chromadb
import chromadb
from chromadb.utils import embedding_functions

1.26.4


In [15]:
# 1. Load your DataFrame (assuming df is already created)
# df has: id | start_time | end_time | text | clean_text

# 2. Initialize Chroma client
client = chromadb.PersistentClient(path="./chroma_db")  # creates/loads DB on disk

# 3. Create or get a collection
collection = client.get_or_create_collection(name="video_transcripts")

# 4. Load SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight, good for transcripts

# 5. Generate embeddings and insert into Chroma
for _, row in df_processed.iterrows():
    embedding = model.encode(row["clean_text"], convert_to_numpy=True)
    #embedding = model.encode(row["clean_text"]).tolist()  # convert to Python list

    metadata = {
        "id": str(row["id"]),
        "start_time": row["start_time"],
        "end_time": row["end_time"],
        "text": row["text"]
    }

    collection.add(
        ids=[str(row["id"])],                  # must be string
        embeddings=[embedding],                # embedding vector
        documents=[row["clean_text"]],         # store clean text
        metadatas=[metadata]                   # store metadata
    )

print(f"Inserted {df_processed.shape[0]} records into Chroma.")


Inserted 252 records into Chroma.


In [ ]:
query = "What April Lyn is talking about?"
results = collection.query(
    query_texts=[query],
    n_results=4
)

from datetime import datetime, timedelta
import math

def round_time_string(time_str: str, kind: str = "start") -> str:
    """
    Round a time string (HH:MM:SS.sss) to floor (start) or ceil (end).

    Args:
        time_str (str): Timestamp like '00:47:14.185'
        kind (str): 'start' -> floor, 'end' -> ceil

    Returns:
        str: Rounded time string in HH:MM:SS format
    """
    # Parse time string
    dt = datetime.strptime(time_str, "%H:%M:%S.%f")

    # Total seconds since midnight
    total_seconds = dt.hour * 3600 + dt.minute * 60 + dt.second + dt.microsecond / 1e6

    if kind == "start":
        rounded_seconds = math.floor(total_seconds)
    elif kind == "end":
        rounded_seconds = math.ceil(total_seconds)
    else:
        raise ValueError("kind must be 'start' or 'end'")

    # Convert back to HH:MM:SS
    rounded_time = str(timedelta(seconds=rounded_seconds))

    # Force HH:MM:SS format with zero padding
    return str(datetime.strptime(rounded_time, "%H:%M:%S").time())

# ✅ Example usage
start_time = "00:47:14.885"
end_time   = "00:47:14.185"
rounded_start = round_time_string(start_time, kind="start")
rounded_end   = round_time_string(end_time, kind="end")
print("Rounded start_time:", rounded_start)  # 00:47:14
print("Rounded end_time:", rounded_end)      # 00:47:15


for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print("Retrieved Text:", doc)
    print("Start Time:", round_time_string(meta['start_time'], 'start'))
    print("Start Time:", round_time_string(meta['end_time'], 'end'))
    print("---")

Rounded start_time: 00:47:14
Rounded end_time: 00:47:15
Retrieved Text: uh, and youtube strategist who's built a name for herself by breaking down what actually works on youtube. so you guys are in for a treat today. so she coins herself as just a girl who loves youtube, but we all know that you're more than that april lyn. um, and today she's gonna be taking us through how
Start Time: 00:00:39
Start Time: 00:00:54
---
Retrieved Text: creator success manager at spotter. we are super, super stoked to have april lynn on today's webinar to teach us how to make a killer intro. so a little bit about april lynn, i'll just, i wrote a little bio about her, so i wanted to share it with you guys, um, to introduce her.
Start Time: 00:00:17
Start Time: 00:00:30
---
Retrieved Text: to share it with you guys, um, to introduce her. and then i'll pass it off to april lynn to take us through what she has for us today. but i'm excited to introduce april lynn, an amazing content creator, uh, and youtube 